## Tools

In [19]:
import { SqlDatabase } from "@langchain/classic/sql_db";
import { DataSource } from "typeorm";

const datasource = new DataSource({
    type: "sqlite",
    database: "./Chinook.db", // Replace with the link to your database
});
const db = await SqlDatabase.fromDataSourceParams({
    appDataSource: datasource,
});

In [20]:
import { tool } from "langchain";
import { z } from "zod";

export const executeSQL = tool(({ query }) => {
    return db.run(query)
}, {
    name: "execute_sql",
    description: "Execute a SQLite command and return results.",
    schema: z.object({ query: z.string() })
})

In [21]:
import { tool } from "langchain";

const FALLBACK_RATES = { USD: 1.0, EUR: 0.92, JPY: 150.0, GBP: 0.79 }

const convertCurrency = tool(async ({ amount, toCurrency }) => {
    try {
        const url = `https://api.exchangerate.host/convert?from=USD&to=${toCurrency}&amount=${amount}`
        const response = await fetch(url, {
            headers: {
                "Accept": "application/json"
            }
        })
        const data = await response.json()
        const result = data.result
        if (result) {
            return round(amount * FALLBACK_RATES[to_currency], 2)
        }
    } catch (error) {
        console.error(`[convert_currency] Falling back due to error: ${error}`)
    }
    if (!(toCurrency in FALLBACK_RATES)) {
        throw new Error("Unsupported currency in fallback mode")
    }
    return round(amount * FALLBACK_RATES[toCurrency], 2)
}, {
    name: "convert_currency",
    description: `
        Convert an amount in USD to another currency using live exchange rates.
        Always use this tool if the user requests a currency different 
        from the one stored in the database (USD).
    `,
    schema: z.object({
        amount: z.number(),
        toCurrency: z.string()
    })
});

In [ ]:
const SYSTEM = `You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool \`execute_sql\` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
`

In [36]:
import "dotenv/config";
import { createAgent } from "langchain";

const agent = createAgent({
    model: "openai:gpt-5",
    tools: [],
    systemPrompt: SYSTEM,
    middleware: []
})

[Module: null prototype] { default: {}, "module.exports": {} }

In [41]:
import { BaseMessage, HumanMessage } from "langchain";

const question = "What is the most costly purchase by Frank Harris in EUR?";
const steps: BaseMessage[] = [];

const stream = await agent.stream({
    messages: [new HumanMessage(question)],
}, {
    streamMode: "values",
});

for await (const message of stream) {
    console.log(message);
    steps.push(message);
}

Error in handler EventStreamCallbackHandler, handleChainStart: TypeError: The stream is closing or is closed.


{
  event: "on_chain_start",
  data: {
    input: {
      messages: [
        HumanMessage {
          "content": "What is the most costly purchase by Frank Harris in EUR?",
          "additional_kwargs": {},
          "response_metadata": {}
        }
      ]
    }
  },
  name: "LangGraph",
  tags: [],
  run_id: "dcb5900c-b325-4093-8c2f-3ae59c6d75ff",
  metadata: {
    langgraph_step: 1,
    langgraph_node: "model_request",
    langgraph_triggers: [ "branch:to:model_request" ],
    langgraph_path: [ "__pregel_pull", "model_request" ],
    langgraph_checkpoint_ns: "model_request:714522ae-cb51-5ce2-b284-8f61e1005e28",
    __pregel_task_id: "714522ae-cb51-5ce2-b284-8f61e1005e28",
    checkpoint_ns: "model_request:714522ae-cb51-5ce2-b284-8f61e1005e28"
  }
}


Error: Abort